<a href="https://colab.research.google.com/github/tanakt-hub/Test/blob/main/JPMA_2022_TF_1_1_demo_(4)%E3%82%A2%E3%83%B3%E3%82%B5%E3%83%B3%E3%83%96%E3%83%AB%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E4%BD%9C%E6%88%90%E3%81%A8%E6%A4%9C%E8%A8%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## Google Driveの接続

In [ ]:
# データ受け渡しのためにGoogle Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# データ保存ディレクトリの指定
datadir = '/content/drive/MyDrive/datadir/'

Mounted at /content/drive


## データのロード

In [ ]:
# 分かち書き済みのテキストとベクトル化したデータを読み込み
import pickle

with open(datadir + 'datadic.pkl', 'rb') as f:
  datadic = pickle.load(f)

## pip install

In [ ]:
!pip install transformers
!pip install mecab-python3 fugashi 
!pip install jaconv neologdn
!pip install sentencepiece
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.3 MB/s 
     |████████████████████████████████| 182 kB 67.5 MB/s 
     |████████████████████████████████| 7.6 MB 54.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 7.2 MB/s 
     |████████████████████████████████| 583 kB 51.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 3.6 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15566 sha256=32c2c8023c29267ef59ea7591c8a9cdc2a9fba382bf1ecbd0323b5eef0c79567
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172971 sha256=d30

## データロードと環境構築

In [ ]:
import jaconv
import unicodedata
import neologdn
import re
import torch

# デバイス設定
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


In [ ]:
# MeCabとNEologdの設定
!apt install mecab libmecab-dev mecab-ipadic-utf8 file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -a -y

# 環境変数でmecabrcの場所を指定
import os
os.environ['MECABRC'] = "/etc/mecabrc" 

# NEologdの展開場所を取得
import subprocess
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_dic_dir_path = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

# 万病辞書のダウンロードと設定
!wget http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic
manbyo_dic_path = 'MANBYO_201907_Dic-utf8.dic'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 11 newly installed, 0 to remove and 5 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-upd

## テストデータの準備

In [ ]:
from sklearn.model_selection import train_test_split

def tts(DataSet, Mode):
  train, test = train_test_split(DataSet, test_size=0.15, random_state = 0)
  if Mode == 'train':
    r = train  # 学習&検証に使用した85%のデータを返す
  elif Mode == 'test':
    r = test  # モデルの学習＆検証に使用しなかった15%を返す

  return r

# アンサンブルモデルの学習/検証モード指定（Appendixの実行時に切り替え）

# 各モデルの学習＆検証に使用したデータを参照する
#Mode = 'train'

# 各モデルの学習＆検証に使用していないデータを参照する
Mode = 'test' 

# 各モデルからPrediction&Probabilityを取得

## sklearn Grid Search Models

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

# 検証対象となるラベル等を取得
label = tts(datadic['flg'], Mode)
texto = tts(datadic['original'], Mode)
textw = tts(datadic['wakati'], Mode)

# 結果格納用のDataFrameを定義
ensumbles = pd.DataFrame(label, columns = ["Label"])
ensumbles["Original Text"] = texto
ensumbles["Wakati Text"] = textw
ensumbles["Negative score"] = 0
ensumbles["Positive score"] = 0

# Grid Searchでモデルを作成したベクトルデータの設定
vecs = [
    'BoW',
    'TFIDF',
    'Emb',
    'Emb_tfidf',
]

# 設定ベクトルに対してループ処理
for vecname in tqdm(vecs, total = len(vecs)):
  X_test = tts(datadic[vecname], Mode) # 対象ベクトルをテストデータとして分割
  pred_prob = 0                              # Probabilityの初期値を0に戻す

  # Grid Search結果の読み込み
  with open(datadir + vecname + '.pkl', 'rb') as f:
    gsmodels = pickle.load(f)

  # Grid Searchを実施した検索対象モデルに対してループ処理
  for clf in tqdm(gsmodels.keys(), total = len(gsmodels.keys())):
    if clf.endswith('_score'):
      # Scoreの結果は使用しないので無視する
      continue

    else:
      # Grid Search結果の最良モデルを読み込み
      bestclfs = gsmodels[clf].best_estimator_

      # Probabilityを取得（predict_probaを持たないモデルは算出）
      if clf in ['RID', 'LSVC']:
        d = bestclfs.decision_function(X_test)
        prob1 = np.exp(d) / np.sum(np.exp(d))
        prob0 = 1 - prob1
        pred_prob += np.stack([prob0,prob1],0).T
      else:
        pred_prob += bestclfs.predict_proba(X_test)

      # PredictionとProbabilityをDataFrameに格納
      predict = pd.DataFrame(bestclfs.predict(X_test), columns=[vecname + '+' + clf])
      probs = pd.DataFrame(pred_prob, columns=[vecname + '+' + clf + '_Neg', vecname + '+' + clf + '_Pos'])

      # 出力用DataFrameに結合
      ensumbles = pd.concat([ensumbles, predict], axis = 1) 
      ensumbles = pd.concat([ensumbles, probs], axis = 1)
      
      # Probabilityの合計値を算出
      ensumbles["Negative score"] += probs[vecname + '+' + clf + '_Neg']
      ensumbles["Positive score"] += probs[vecname + '+' + clf + '_Pos'] 

# 結果の確認
display(ensumbles)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

,Label,Original Text,Wakati Text,Negative score,Positive score,BoW+LR,BoW+LR_Neg,BoW+LR_Pos,BoW+RID,BoW+RID_Neg,...,Emb_tfidf+MLP1_Pos,Emb_tfidf+MLP2,Emb_tfidf+MLP2_Neg,Emb_tfidf+MLP2_Pos,Emb_tfidf+AB1,Emb_tfidf+AB1_Neg,Emb_tfidf+AB1_Pos,Emb_tfidf+AB2,Emb_tfidf+AB2_Neg,Emb_tfidf+AB2_Pos
0,1,心尖部血栓を認めた。,心尖部血栓 認める た,153.880774,210.119226,1,0.081690,0.918310,1,1.080023,...,7.406055,1,2.594802,8.405198,1,2.853595,9.146405,1,2.888054,10.111946
1,1,2022/05/30、患者は劇症型心筋炎を発現した。,0 0 0 患者 劇症型 心筋炎 発現 する た,111.154445,252.845555,1,0.004947,0.995053,1,1.002480,...,7.103408,1,2.906669,8.093331,1,3.114317,8.885683,1,3.140349,9.859651
2,1,"再度診察を行ったところ甲状腺左築に圧痛があり,追加検査にてTSH0.006μU/mL,Fre...",診察 行う た ところ 甲状腺 左 築 圧痛 ある 追加 検査 TSH 0 0 μ U mL...,116.752203,247.247797,1,0.000018,0.999982,1,0.995012,...,6.581354,1,3.513339,7.486661,1,3.831085,8.168915,1,3.980182,9.019818
3,0,頭部CTは異常所見なく、頭痛の器質的疾患は否定された。,頭部 C，T 異常所 見る ない 頭痛 の 器質的疾患 否定 する れる た,257.772158,106.227842,0,0.979947,0.020053,0,1.979491,...,6.103686,1,4.177022,6.822978,1,4.618394,7.381606,1,5.057396,7.942604
4,1,2021/07/24、両大腿の感覚障害と歩行障害(バランスがとれず、スムーズに一歩ずつ足が出...,0 0 0 大腿 感覚障害 歩行障害 バランス とれる ぬ スムーズ 一 歩 足 出る ない...,116.318061,247.681939,1,0.010987,0.989013,1,1.008804,...,6.911332,1,3.199668,7.800332,1,3.464841,8.535159,1,3.536519,9.463481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,1,翌日に右眼の視力低下を自覚した。,翌日 右 眼 視力低下 自覚 する た,125.356967,238.643033,1,0.000040,0.999960,1,0.996732,...,6.480464,1,3.566789,7.433211,1,3.803968,8.196032,1,3.967592,9.032408
646,0,被験者にSARS-CoV2感染中に悪化した基礎疾患はなかった。,被験者 SARS CoV 0 感染中 悪化 する た 基礎疾患 ない た,335.147568,28.852432,0,0.989795,0.010205,0,1.989298,...,0.618357,0,10.370103,0.629897,0,11.047606,0.952394,0,11.977554,1.022446
647,1,両側肺動脈に透亮像を認めた。,両側肺 動脈 透亮像 認める た,127.971956,236.028044,1,0.030893,0.969107,1,1.028945,...,6.730072,1,3.275509,7.724491,1,3.578412,8.421588,1,3.627154,9.372846
648,0,ワクチン接種以来、COVID-19の検査はしていない。,ワクチン接種 以来 COVID 0 検査 する いる ない,341.279749,22.720251,0,0.993924,0.006076,0,1.993478,...,0.497064,0,10.491369,0.508631,0,11.179360,0.820640,0,12.141780,0.858220


## BERT Models

In [ ]:
# 評価関数の定義
def eval(eval_dataloader, model, device, tqdm):
  logits     = []
  inputs     = []

  model.eval()    # 検証モード
  for n_iter, d in tqdm(enumerate(eval_dataloader), total=len(eval_dataloader)):
    with torch.no_grad():
      outputs = model(
        d[0].to(device),                    # input_ids_t
        attention_mask = d[1].to(device),   # attention_masks_t
        token_type_ids=None
        )
    
    logits += outputs.logits.sigmoid().cpu().tolist()
    inputs += d[0].tolist()
  
  eval_res = pd.DataFrame(logits, columns=['logit0', 'logit1'])
  eval_pred = np.argmax(eval_res.values, axis=1).tolist()
  eval_res['pred']  = eval_pred
  eval_res['input_ids']  = inputs

  return eval_res

In [ ]:
from transformers import  BertJapaneseTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader, TensorDataset

# BERT Prediction関数の定義
def bertprediction(model_name, text, label):
  # 事前学習モデルの読み込み
  model = BertForSequenceClassification.from_pretrained(
      model_name,                    # 読み込むモデル名
      num_labels = 2,                # Binary classification
      output_attentions = False,     # Attentionの出力
      output_hidden_states = False,  # 隠れ層の出力
      )
  model.to(device)

  # 事前学習モデルのトークナイザ設定
  MeCabDic = {"mecab_dic": None, "mecab_option": "-d " + neologd_dic_dir_path + " -u " + manbyo_dic_path}

  tokenizer = BertJapaneseTokenizer.from_pretrained(
      model_name,
      word_tokenizer_type = "mecab",
      mecab_kwargs = MeCabDic
      )

  # 最大トークン数の設定
  # Fine-tuningしたBERTモデルの最大長に合わせる
  model_seq_len = 512

  # テストデータが最大長に満たない場合は計算量削減のためテストデータの最大長に切り落とし
  max_tk = 0
  for i, chktoken in enumerate(text):
    tk = tokenizer.tokenize(chktoken)
    if len(tk) > max_tk:
      max_tk = len(tk)
      id = i

  max_len = max_tk + 2 if max_tk + 2 < model_seq_len else model_seq_len

  # 最大長データの確認
  tokchk = tokenizer.encode_plus(
      text[id],
      add_special_tokens = True,        # スペシャルトークンの追加
      truncation = True,                # モデル定義長を超える場合の切り捨て
      max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
      padding = 'max_length',           # 最大長までPADDING
      return_overflowing_tokens = True, # 切り捨てられたトークンを返す
      num_truncated_tokens = True       # 切り捨てられたトークン数を返す
      )

  print("最大トークン数:", max_tk)
  print("*** 最大トークン数に分割されるテキスト ***")
  print("  ", text[id])
  print("*** BERTに入力されるテキスト ***")
  print("  ", tokenizer.decode(tokchk['input_ids']))
  print("*** 切り捨てられるテキスト ***")
  print("  ", tokenizer.decode(tokchk['overflowing_tokens']))

  # トークナイズ処理
  # 必要なToken IDとAttentionマスクを取得
  token_ids = []
  attention_masks = []

  for t in text:
    tknzd = tokenizer.encode_plus(
        t,
        add_special_tokens = True,        # スペシャルトークンの追加
        truncation = True,                # モデル定義長を超える場合の切り捨て
        max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
        padding = 'max_length'            # 最大長までPADDING
        )
    token_ids.append(tknzd['input_ids'])
    attention_masks.append(tknzd['attention_mask'])

  # tensor型に変換
  token_ids_t = torch.tensor(token_ids)
  attention_masks_t = torch.tensor(attention_masks)

  # データセットとデータローダーの作成
  eval_dataset = TensorDataset(token_ids_t, attention_masks_t)
  eval_dataloader = DataLoader(
      eval_dataset,
      batch_size = 128,
      shuffle = False,
      drop_last = False
      )

  # Predictionの実行
  out = eval(eval_dataloader, model, device, tqdm)

  out['Text'] = [t.strip('[CLS] [SEP] [PAD]') for t in tokenizer.batch_decode(out['input_ids'])]
  out['text1'] = text
  labeldf = pd.DataFrame(label, columns = ["Label"])

  out = pd.concat([labeldf, out], axis = 1 )

  return out

In [ ]:
# Fine-tuning済みのモデルパスを指定
models = [
  ('BERT+BERT0', '/content/drive/MyDrive/datadir/bert/BERT_MODEL_0'),
  ('BERT+BERT1', '/content/drive/MyDrive/datadir/bert/BERT_MODEL_1'),
  ('BERT+BERT2', '/content/drive/MyDrive/datadir/bert/BERT_MODEL_2'),
  ('BERT+BERT3', '/content/drive/MyDrive/datadir/bert/BERT_MODEL_3'),
  ('BERT+BERT4', '/content/drive/MyDrive/datadir/bert/BERT_MODEL_4'),
]

# BERTモデルのPredictionとProbabilityを取得
for name, model in tqdm(models, total= len(models)):
  df = bertprediction(model, textw, label)

  df = df.rename(columns={'pred': name, 'logit0': name + '_Neg', 'logit1': name + '_Pos'})

  ensumbles = pd.concat([ensumbles, df[name]], axis = 1)
  ensumbles = pd.concat([ensumbles, df[name + '_Neg']], axis = 1)
  ensumbles = pd.concat([ensumbles, df[name + '_Pos']], axis = 1)
  ensumbles["Negative score"] += df[name + '_Neg']
  ensumbles["Positive score"] += df[name + '_Pos']


  0%|          | 0/5 [00:00<?, ?it/s]

Keyword arguments {'num_truncated_tokens': True} not recognized.


最大トークン数: 105
*** 最大トークン数に分割されるテキスト ***
   ラフチジン 錠 0m g サワイ 0 錠 0 朝食 夕食 後 0 日間 ベイスン OD 錠 0 0 0 0m g 0 錠 0 朝食 直前 0 日間 フェブリク 錠 0m g 0 錠 0 朝食 後 0 日間 アダラート CR 錠 0m g 0 錠 0 夕食 後 0 日間 スピロノラクトン 錠 0m g TOWA 0 0 錠 ラシックス 錠 0m g 0 錠 0 朝食 後 0 日間 ノイエル カプセル 0m g 0 カプセル 0 朝食 夕食 後 0 日間
*** BERTに入力されるテキスト ***
   [CLS] ラフチジン 錠 0m g サワイ 0 錠 0 朝食 夕食 後 0 日間 ベイスン OD 錠 0 0 0 0m g 0 錠 0 朝食 直前 0 日間 フェブリク 錠 0m g 0 錠 0 朝食 後 0 日間 アダラート CR 錠 0m g 0 錠 0 夕食 後 0 日間 スピロノラクトン 錠 0m g TOWA 0 0 錠 ラシックス 錠 0m g 0 錠 0 朝食 後 0 日間 ノイエル カプセル 0m g 0 カプセル 0 朝食 夕食 後 0 日間 [SEP]
*** 切り捨てられるテキスト ***
   


  0%|          | 0/6 [00:00<?, ?it/s]

Keyword arguments {'num_truncated_tokens': True} not recognized.


...

最大トークン数: 105
*** 最大トークン数に分割されるテキスト ***
   ラフチジン 錠 0m g サワイ 0 錠 0 朝食 夕食 後 0 日間 ベイスン OD 錠 0 0 0 0m g 0 錠 0 朝食 直前 0 日間 フェブリク 錠 0m g 0 錠 0 朝食 後 0 日間 アダラート CR 錠 0m g 0 錠 0 夕食 後 0 日間 スピロノラクトン 錠 0m g TOWA 0 0 錠 ラシックス 錠 0m g 0 錠 0 朝食 後 0 日間 ノイエル カプセル 0m g 0 カプセル 0 朝食 夕食 後 0 日間
*** BERTに入力されるテキスト ***
   [CLS] ラフチジン 錠 0m g サワイ 0 錠 0 朝食 夕食 後 0 日間 ベイスン OD 錠 0 0 0 0m g 0 錠 0 朝食 直前 0 日間 フェブリク 錠 0m g 0 錠 0 朝食 後 0 日間 アダラート CR 錠 0m g 0 錠 0 夕食 後 0 日間 スピロノラクトン 錠 0m g TOWA 0 0 錠 ラシックス 錠 0m g 0 錠 0 朝食 後 0 日間 ノイエル カプセル 0m g 0 カプセル 0 朝食 夕食 後 0 日間 [SEP]
*** 切り捨てられるテキスト ***
   


  0%|          | 0/6 [00:00<?, ?it/s]

Keyword arguments {'num_truncated_tokens': True} not recognized.


最大トークン数: 105
*** 最大トークン数に分割されるテキスト ***
   ラフチジン 錠 0m g サワイ 0 錠 0 朝食 夕食 後 0 日間 ベイスン OD 錠 0 0 0 0m g 0 錠 0 朝食 直前 0 日間 フェブリク 錠 0m g 0 錠 0 朝食 後 0 日間 アダラート CR 錠 0m g 0 錠 0 夕食 後 0 日間 スピロノラクトン 錠 0m g TOWA 0 0 錠 ラシックス 錠 0m g 0 錠 0 朝食 後 0 日間 ノイエル カプセル 0m g 0 カプセル 0 朝食 夕食 後 0 日間
*** BERTに入力されるテキスト ***
   [CLS] ラフチジン 錠 0m g サワイ 0 錠 0 朝食 夕食 後 0 日間 ベイスン OD 錠 0 0 0 0m g 0 錠 0 朝食 直前 0 日間 フェブリク 錠 0m g 0 錠 0 朝食 後 0 日間 アダラート CR 錠 0m g 0 錠 0 夕食 後 0 日間 スピロノラクトン 錠 0m g TOWA 0 0 錠 ラシックス 錠 0m g 0 錠 0 朝食 後 0 日間 ノイエル カプセル 0m g 0 カプセル 0 朝食 夕食 後 0 日間 [SEP]
*** 切り捨てられるテキスト ***
   


  0%|          | 0/6 [00:00<?, ?it/s]

## 各モデルの評価スコアを計算

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# 実行結果を保存したensumblesを読みこんで正解ラベルと各Predictionを比較する
x = []
for c in ensumbles.columns[5:]:
  if not c.endswith(('_Pos','_Neg')):
    y = []
    y.append(str(c).split('+')[0])
    try:
      y.append(str(c).split('+')[1])
    except:
      y.append('')

    y.append(f1_score(ensumbles["Label"], ensumbles[c].astype(int)))
    y.append(accuracy_score(ensumbles["Label"], ensumbles[c].astype(int)))
    y.append(recall_score(ensumbles["Label"], ensumbles[c].astype(int)))
    y.append(precision_score(ensumbles["Label"], ensumbles[c].astype(int)))
    x.append(y)

  else:
    continue

scores = pd.DataFrame(x, columns = ["Vector", "Method", "F1", "Accuracy", "Recall", "Precision"])
scores["idx"] = scores["Vector"]+"_"+scores["Method"]
scores.set_index("idx",inplace=True)

# 結果と評価スコアをCSVで保存
ensumbles.to_csv(datadir + 'ensumbles_' + Mode + '.csv')
scores.to_csv(datadir + 'scores_' + Mode + '.csv')


# アンサンブル

## アンサンブルモデルの選択

In [ ]:
import re

# Gred searchで分割検索したモデルがあるため、モデル名から分割数字を削除
selection = scores.copy()
selection["Method_"] = selection["Method"].apply(lambda x: re.sub("[0-9]","",x))

# グループ化で最大スコアのみ抽出する
max_idx = selection.groupby(["Vector","Method_"])["F1"].idxmax()
selection = selection[selection.index.isin(max_idx)]

# 複合キーをIndexに再設定
selection["idx"] = selection["Vector"]+"_"+selection["Method_"]
selection.set_index("idx",inplace=True)


In [ ]:
import pulp

# 以下のように最適なモデルを選択する
#     目的：選択したモデルの合計F1スコアが最も高くなるようにする
#   制約 1：各Vectorごとにスコアの良い手法を1つ選択する
#   制約 2：手法は重複しないように選択する

# 最適化問題の作成
problem = pulp.LpProblem('BestModels', pulp.LpMaximize)

# 評価に使用したVectorと手法のリスト
V = set(selection["Vector"])
M = set(selection["Method_"])

# 最適化変数の定義
x = pulp.LpVariable.dicts('x', [(v,m) for v in V for m in M], cat='Binary')

# 係数とするF1スコアの取得
def F1values(Tgt):
  v = Tgt[0]
  m = Tgt[1]
  try:
    x = selection.at[v+"_"+m,"F1"]
  except:
    x = 0
  return x

f1 = {}
for i in x.keys():
  f1[i] = F1values(i)

# 目的関数の定義
problem += pulp.lpSum( [f1[i] * x[i] for i in x.keys()] )   # F1スコア * 組み合わせの合計値

# 制約の定義
for v in V:
  problem += pulp.lpSum([x[v,m] for m in M]) == 1  # 各Vectorから1つずつ選択されること

for m in M:
  problem += pulp.lpSum([x[v,m] for v in V]) <= 1  # 選択された手法が重複しないこと

# 最適化Statusの確認
result = problem.solve()
print('Resolution status:', pulp.LpStatus[result])

# 最適な組み合わせの出力
result = {}
for v in V:
  result[v] = [m for m in M if x[v,m].value()==1]
print('最適な組み合わせ:', result)

# 選択されたモデルのスコアを確認
selected = []
for v in result.keys():
  selected.append(v + "_" + selection.at[v + "_" + result[v][0], "Method"])
scores[scores.index.isin(selected)]

Resolution status: Optimal
最適な組み合わせ: {'Emb_tfidf': ['MLP'], 'BoW': ['LSVC'], 'Emb': ['LR'], 'BERT': ['BERT'], 'TFIDF': ['RID']}


,Vector,Method,F1,Accuracy,Recall,Precision
idx,,,,,,
BoW_LSVC,BoW,LSVC,0.924419,0.920000,0.913793,0.935294
TFIDF_RID,TFIDF,RID,0.931429,0.926154,0.936782,0.926136
Emb_LR,Emb,LR,0.905386,0.900000,0.893678,0.917404
Emb_tfidf_MLP2,Emb_tfidf,MLP2,0.914863,0.909231,0.910920,0.918841
BERT_BERT3,BERT,BERT3,0.944444,0.941538,0.928161,0.961310


## 選択モデルの結果抽出とアンサンブル

In [ ]:
import pandas as pd
#mode= 'train'
#mode= 'valid'
ensumbles = pd.read_csv(datadir + 'ensumbles_' + Mode + '.csv')


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# 必要なカラムを選択
cols_h = [
  "Label",
  "Original Text",
  "Wakati Text", 
  ]

cols_d = []
for v in result.keys():
  cols_d.append(v + "+" + selection.at[v + "_" + result[v][0], "Method"])

ensumbles_d = ensumbles[cols_h+cols_d]

# いずれか一つのモデルがPositiveと判定している場合はPositiveとする
vres = pd.DataFrame(ensumbles_d[cols_d].sum(axis=1), columns=["ens_v"])
ensumbles_d = pd.concat([ensumbles_d, vres], axis = 1) 
ensumbles_d["ens_v"] = ensumbles_d["ens_v"].apply(lambda x: 0 if x == 0 else 1)

# 結果の確認
display(ensumbles_d)

print('****: Ensumble Scores')
print('        F1:', f1_score(ensumbles_d["Label"], ensumbles_d["ens_v"]))
print('    Recall:', recall_score(ensumbles_d["Label"], ensumbles_d["ens_v"]))
print('  Accuracy:', accuracy_score(ensumbles_d["Label"], ensumbles_d["ens_v"]))
print(' Precision:', precision_score(ensumbles_d["Label"], ensumbles_d["ens_v"]))

,Label,Original Text,Wakati Text,Emb_tfidf+MLP2,BoW+LSVC,Emb+LR,BERT+BERT3,TFIDF+RID,ens_v
0,1,心尖部血栓を認めた。,心尖部血栓 認める た,1,1,1,1,1,1
1,1,2022/05/30、患者は劇症型心筋炎を発現した。,0 0 0 患者 劇症型 心筋炎 発現 する た,1,1,1,1,1,1
2,1,"再度診察を行ったところ甲状腺左築に圧痛があり,追加検査にてTSH0.006μU/mL,Fre...",診察 行う た ところ 甲状腺 左 築 圧痛 ある 追加 検査 TSH 0 0 μ U mL...,1,1,1,1,1,1
3,0,頭部CTは異常所見なく、頭痛の器質的疾患は否定された。,頭部 C，T 異常所 見る ない 頭痛 の 器質的疾患 否定 する れる た,1,0,1,0,0,1
4,1,2021/07/24、両大腿の感覚障害と歩行障害(バランスがとれず、スムーズに一歩ずつ足が出...,0 0 0 大腿 感覚障害 歩行障害 バランス とれる ぬ スムーズ 一 歩 足 出る ない...,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
645,1,翌日に右眼の視力低下を自覚した。,翌日 右 眼 視力低下 自覚 する た,1,1,1,1,1,1
646,0,被験者にSARS-CoV2感染中に悪化した基礎疾患はなかった。,被験者 SARS CoV 0 感染中 悪化 する た 基礎疾患 ない た,0,0,0,0,0,0
647,1,両側肺動脈に透亮像を認めた。,両側肺 動脈 透亮像 認める た,1,1,1,1,1,1
648,0,ワクチン接種以来、COVID-19の検査はしていない。,ワクチン接種 以来 COVID 0 検査 する いる ない,0,0,0,0,0,0


****: Ensumble Scores
        F1: 0.9083665338645418
    Recall: 0.9827586206896551
  Accuracy: 0.8938461538461538
 Precision: 0.8444444444444444


# Appendix: Stacking

## Stacking

In [ ]:
# 選択モデルのProbability値を用いた新たなモデルを構築する
# タスクの目的を考慮するとRecall値を重視した上記アンサンブルモデルが良いと思われる
# 実行せずとも本編に影響なし
# 実行には「テストデータの準備」セクションで train モードを選択し、モデル学習に使用したデータのProbabilityを用いたPredictionモデルを作成する必要がある

import pandas as pd
from sklearn.linear_model import RidgeClassifier
import pickle

#Mode ='train'
#Mode ='test'

predscores = pd.read_csv(datadir + 'ensumbles_' + Mode + '.csv')

# Labelと使用するモデルを指定
kcol = (
  "Label",
  "Emb+LR",
  "BoW+LSVC",
  "Emb_tfidf+MLP2",
  "TFIDF+RID"
)

predscores_d = predscores
for c in predscores.columns:
  if c.startswith(kcol) == False:
    predscores_d = predscores_d.drop(columns = [c])

predscores_d_label = predscores_d["Label"]
predscores_d_logit = predscores_d.drop(columns = ["Label"])
for c in predscores_d_logit.columns:
  if c.endswith(('_Pos','_Neg')) == False:
    predscores_d_logit = predscores_d_logit.drop(columns = [c])

# Trainモードの場合はリッジ回帰のモデルを作成して保存する
if Mode == 'train':
  print('*** Train mode execution ***')
  ridge = RidgeClassifier(alpha=10).fit(predscores_d_logit, predscores_d_label)

  sres = pd.DataFrame(ridge.predict(predscores_d_logit), columns=["ens_s"])
  ensumbles_d = pd.concat([ensumbles_d, sres], axis = 1) 

  with open(datadir + 'logitensumbles.pkl', 'wb') as f:
    pickle.dump(ridge, f)

# Testモードの場合は保存したモデルを読み込んで予測結果を返す
elif Mode == 'test':
  print('*** Test mode execution ***')
  with open(datadir + 'logitensumbles.pkl', 'rb') as f:
    ridge = pickle.load(f)

  sres = pd.DataFrame(ridge.predict(predscores_d_logit), columns=["ens_s"])
  ensumbles_d = pd.concat([ensumbles_d, sres], axis = 1) 


*** Talid mode execution ***


In [ ]:
# Predictionスコアの確認
from sklearn.metrics import f1_score, accuracy_score, recall_score
print('****: vote scores')
print('      f1:', f1_score(ensumbles_d["Label"], ensumbles_d["ens_v"]))
print('  recall:', recall_score(ensumbles_d["Label"], ensumbles_d["ens_v"]))
print('\n****: stacking scores')
print('      f1:', f1_score(ensumbles_d["Label"], ensumbles_d["ens_s"]))
print('  recall:',recall_score(ensumbles_d["Label"], ensumbles_d["ens_s"]))


ensumbles_d.to_csv(datadir + 'ensumbles_d_' + Mode + '.csv')
predscores_d.to_csv(datadir + 'predscores_d_' + Mode + '.csv')

****: vote scores
      f1: 0.9083665338645418
  recall: 0.9827586206896551

****: stacking scores
      f1: 0.9224011713030746
  recall: 0.9051724137931034


## Grid search

In [ ]:
# StackingモデルのGrid Searchを試す

import pandas as pd
Mode ='train'
ensumbles = pd.read_csv(datadir + 'ensumbles_' + Mode + '.csv')

# Labelと使用するモデルを指定
kcol = (
  "Label",
  "Emb+LR",
  "BoW+LSVC",
  "Emb_tfidf+MLP2",
  "TFIDF+RID"
)
predscores_d = ensumbles
for c in ensumbles.columns:
  if c.startswith(kcol) == False:
    predscores_d = predscores_d.drop(columns = [c])
predscores_d

,Label,BoW+LSVC,BoW+LSVC_Neg,BoW+LSVC_Pos,TFIDF+RID,TFIDF+RID_Neg,TFIDF+RID_Pos,Emb+LR,Emb+LR_Neg,Emb+LR_Pos,Emb_tfidf+MLP2,Emb_tfidf+MLP2_Neg,Emb_tfidf+MLP2_Pos
0,0,0,2.865483,0.134517,0,1.804522,0.195478,0,0.705125,0.294875,0,9.286138,1.713862
1,1,1,2.000010,0.999990,1,1.046517,0.953483,1,0.017963,0.982037,1,2.548988,8.451012
2,0,0,2.973476,0.026524,0,1.960641,0.039359,0,0.834273,0.165727,0,10.219468,0.780532
3,1,1,2.103236,0.896764,1,1.232642,0.767358,0,0.637467,0.362533,1,3.028278,7.971722
4,1,1,1.998774,1.001226,1,1.034882,0.965118,1,0.000420,0.999580,1,3.544697,7.455303
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3673,0,0,2.966331,0.033669,0,1.897143,0.102857,0,0.794448,0.205552,0,9.584764,1.415236
3674,1,1,2.104296,0.895704,1,1.197528,0.802472,0,0.501464,0.498536,1,2.275291,8.724709
3675,0,0,2.994057,0.005943,0,1.955861,0.044139,0,0.962134,0.037866,0,10.666138,0.333862
3676,1,1,2.120427,0.879573,1,1.363392,0.636608,0,0.878684,0.121316,1,4.188953,6.811047


In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import LinearSVC

# 線形回帰モデルの Gred search を実行

clf_LR = LogisticRegression(max_iter=50000)
clf_RID = RidgeClassifier(max_iter=50000)
clf_LSVC = LinearSVC(max_iter=50000)

param_LR = {
    'C': [10**i for i in range(-5,5)]
     }

param_RID = {
    'alpha':  [10**i for i in range(-5,5)]
     }

param_LSVC = {
    'C': [10**i for i in range(-5,5)]
     }

clfs = {
    "LR": [clf_LR, param_LR],
    "RID": [clf_RID, param_RID],
    "LSVC": [clf_LSVC, param_LSVC], 
  }

def grid_search(clf, param, Data, Label):
  grid_search = GridSearchCV(
      clf,
      param,
      cv=5,
      scoring='f1') # f1でスコアリング
  return grid_search.fit(Data, Label)

predscores_d_label = predscores_d["Label"]
predscores_d_logit = predscores_d.drop(columns = ["Label"])
for c in predscores_d_logit.columns:
  if c.endswith(('_Pos','_Neg')) == False:
    predscores_d_logit = predscores_d_logit.drop(columns = [c])


X_train, X_test, y_train, y_test = train_test_split(predscores_d_logit, predscores_d_label, test_size=0.15, random_state = 0)

for clfname in clfs.keys():

  print('  *** classifier:', clfname)
  gs = grid_search(clfs[clfname][0], clfs[clfname][1], X_train, y_train)

  print('****', clfname)
  print(gs.score(X_test, y_test))
  display(pd.DataFrame(gs.cv_results_))


  *** classifier: LR
**** LR
0.998324958123953


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012448,0.001631,0.003924,0.002142,0.00001,{'C': 1e-05},0.961877,0.954745,0.943560,0.954745,0.948905,0.952766,0.006172,10
1,0.014605,0.001104,0.002924,0.000087,0.0001,{'C': 0.0001},0.971514,0.963855,0.956652,0.957576,0.975904,0.965100,0.007582,9
2,0.015372,0.001209,0.003631,0.001213,0.001,{'C': 0.001},0.972973,0.968134,0.955090,0.960366,0.981818,0.967676,0.009382,8
3,0.021564,0.003008,0.002950,0.000109,0.01,{'C': 0.01},0.987805,0.978528,0.974203,0.964778,0.989313,0.978925,0.009048,7
4,0.028212,0.006482,0.003135,0.000154,0.1,{'C': 0.1},0.998473,0.998473,0.995406,0.992343,0.998473,0.996634,0.002452,6
5,0.035313,0.008108,0.003055,0.000102,1,{'C': 1},1.000000,0.998473,0.998473,0.998473,0.996951,0.998474,0.000964,5
6,0.035229,0.009964,0.002940,0.000108,10,{'C': 10},1.000000,1.000000,1.000000,0.998473,0.996951,0.999085,0.001220,3
7,0.036720,0.001797,0.003057,0.000327,100,{'C': 100},1.000000,1.000000,1.000000,0.998473,0.996951,0.999085,0.001220,3
8,0.042629,0.005021,0.002940,0.000192,1000,{'C': 1000},1.000000,1.000000,1.000000,1.000000,0.996951,0.999390,0.001220,1
9,0.036661,0.006897,0.003793,0.001784,10000,{'C': 10000},1.000000,1.000000,1.000000,1.000000,0.996951,0.999390,0.001220,1


  *** classifier: RID
**** RID
0.998324958123953


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004949,0.000370,0.002919,0.000193,0.00001,{'alpha': 1e-05},1.000000,0.998473,1.000000,0.998478,0.996951,0.998780,0.001141,1
1,0.004633,0.000510,0.002846,0.000127,0.0001,{'alpha': 0.0001},1.000000,0.998473,1.000000,0.998478,0.996951,0.998780,0.001141,1
2,0.004354,0.000179,0.002826,0.000085,0.001,{'alpha': 0.001},1.000000,0.998473,1.000000,0.998478,0.996951,0.998780,0.001141,1
3,0.004407,0.000069,0.002845,0.000104,0.01,{'alpha': 0.01},1.000000,0.998473,1.000000,0.998478,0.996951,0.998780,0.001141,1
4,0.005337,0.001473,0.002944,0.000071,0.1,{'alpha': 0.1},1.000000,0.998473,1.000000,0.998478,0.996951,0.998780,0.001141,1
5,0.005164,0.001268,0.002940,0.000087,1,{'alpha': 1},1.000000,0.998473,1.000000,0.998478,0.996951,0.998780,0.001141,1
6,0.004433,0.000177,0.002902,0.000070,10,{'alpha': 10},1.000000,0.998473,1.000000,0.998478,0.996951,0.998780,0.001141,1
7,0.004399,0.000044,0.002897,0.000134,100,{'alpha': 100},1.000000,0.998473,1.000000,0.998473,0.996951,0.998780,0.001141,8
8,0.004545,0.000123,0.002922,0.000062,1000,{'alpha': 1000},0.992413,0.987805,0.978788,0.974203,0.987805,0.984203,0.006673,9
9,0.004467,0.000099,0.002929,0.000048,10000,{'alpha': 10000},0.972973,0.966767,0.955090,0.961948,0.980333,0.967422,0.008718,10


  *** classifier: LSVC
**** LSVC
1.0


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006107,0.002267,0.002957,0.000383,0.00001,{'C': 1e-05},0.971429,0.966667,0.955090,0.960366,0.981818,0.967074,0.009221,10
1,0.004490,0.000084,0.002704,0.000197,0.0001,{'C': 0.0001},0.974436,0.968134,0.955090,0.960366,0.981818,0.967969,0.009564,9
2,0.002841,0.000126,0.001928,0.000107,0.001,{'C': 0.001},0.996951,0.990769,0.983155,0.980031,0.992343,0.988650,0.006189,8
3,0.003013,0.000115,0.001965,0.000111,0.01,{'C': 0.01},1.000000,0.998473,0.998473,0.998473,0.996951,0.998474,0.000964,7
4,0.004989,0.001345,0.002224,0.000331,0.1,{'C': 0.1},1.000000,0.998473,1.000000,0.998473,0.996951,0.998780,0.001141,6
5,0.008976,0.001954,0.002253,0.000541,1,{'C': 1},1.000000,1.000000,1.000000,1.000000,0.996951,0.999390,0.001220,1
6,0.028124,0.008743,0.002374,0.000333,10,{'C': 10},1.000000,1.000000,1.000000,1.000000,0.996951,0.999390,0.001220,1
7,0.020767,0.002536,0.002113,0.000248,100,{'C': 100},1.000000,1.000000,1.000000,1.000000,0.996951,0.999390,0.001220,1
8,0.020890,0.001178,0.002226,0.000255,1000,{'C': 1000},1.000000,1.000000,1.000000,0.998473,0.996951,0.999085,0.001220,4
9,0.019833,0.001742,0.002091,0.000183,10000,{'C': 10000},1.000000,1.000000,1.000000,0.998473,0.996951,0.999085,0.001220,4
